In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [41]:
FullLabelPath = 'parelsnoer/Labels_parelsnoer_annotated_extensive3_pruned.txt'
FullLabelFile = pd.read_csv(FullLabelPath,names=['ID','label','extra'],sep='\t',low_memory=False)

In [42]:
FullLabelFile

,ID,label,extra
0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0
1,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0
2,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0
3,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0
4,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0
...,...,...,...
35455,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0
35456,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0
35457,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0
35458,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0


In [43]:
FullLabelFile['label'].value_counts()

label
14    9600
0     9195
7     7350
15    5235
3     2475
13     990
12     495
5       75
1       45
Name: count, dtype: int64

In [44]:
amountsPerClass = [1, 5, 10, 20]

In [45]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s', 1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES', 'NIFTI') + '.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath': NiftiPath, 'slicenum': slicenum})

In [46]:
FilepathsAndSlicenums = FullLabelFile.apply(extractNiftiFilepathAndSlicenum, axis=1)

In [47]:
FullLabelFile = FullLabelFile.merge(FilepathsAndSlicenums, left_index=True, right_index=True)

In [48]:
FullLabelFile

,ID,label,extra,NiftiPath,slicenum
0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,17
1,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,5
2,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,6
3,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,12
4,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,18
...,...,...,...,...,...
35455,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,11
35456,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,19
35457,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,8
35458,/data/scratch/r098375/data/parelsnoer_new/NIFT...,15,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,9


In [49]:
for numTrainSamples in amountsPerClass:
    extension = os.path.splitext(FullLabelPath)
    basename = extension[0]
    extension = extension[1]
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()
    for label in FullLabelFile['label'].unique():
        outname_train = f'{basename}_s{numTrainSamples}_train{extension}'
        outname_test = f'{basename}_s{numTrainSamples}_test{extension}'
        LabelFrame = FullLabelFile[FullLabelFile['label']==label]
        NiftiNames = LabelFrame['NiftiPath'].unique()
        try:
            train, test = train_test_split(NiftiNames, train_size=numTrainSamples, shuffle=True, random_state=42)
        except:
            train = NiftiNames
            test = []
        for tr in train:
            if df_train.empty:
                df_train = LabelFrame[LabelFrame['NiftiPath']==tr]
            else:
                temp = LabelFrame[LabelFrame['NiftiPath']==tr]
                df_train = pd.concat([df_train, temp],axis=0)
        for te in test:
            if df_test.empty:
                df_test = LabelFrame[LabelFrame['NiftiPath']==te]
            else:
                temp = LabelFrame[LabelFrame['NiftiPath']==te]
                df_test = pd.concat([df_test, temp],axis=0)
        df_train[['ID','label','extra']].to_csv(outname_train, index=False, header=False, sep='\t')
        df_test[['ID','label','extra']].to_csv(outname_test, index=False, header=False, sep='\t')

In [50]:
df_train

,ID,label,extra,NiftiPath,slicenum
9630,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,8
9631,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7
9632,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,16
9633,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,10
9634,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,6
...,...,...,...,...,...
16270,/data/scratch/r098375/data/parelsnoer_new/NIFT...,1,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,17
16271,/data/scratch/r098375/data/parelsnoer_new/NIFT...,1,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,18
16272,/data/scratch/r098375/data/parelsnoer_new/NIFT...,1,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7
16273,/data/scratch/r098375/data/parelsnoer_new/NIFT...,1,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,11


In [51]:
df_test

,ID,label,extra,NiftiPath,slicenum
32895,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,8
32896,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,19
32897,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,6
32898,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,10
32899,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,17
...,...,...,...,...,...
26680,/data/scratch/r098375/data/parelsnoer_new/NIFT...,13,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,13
26681,/data/scratch/r098375/data/parelsnoer_new/NIFT...,13,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,9
26682,/data/scratch/r098375/data/parelsnoer_new/NIFT...,13,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,12
26683,/data/scratch/r098375/data/parelsnoer_new/NIFT...,13,0,/data/scratch/r098375/data/parelsnoer_new/NIFT...,7
